In [29]:
#import
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import pandas as pd
import regex as re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess
import json

class HealthSpider(scrapy.Spider):
    name = 'health'
    start_urls = ['https://www.webteb.com/diseases']

    def parse(self, response):
        items = response.css('div.content.acc-content ul.list li')
        for item in items:
            disease_type = item.css('a::text').get().strip()
            disease_link = item.css('a::attr(href)').get()
            base_link = disease_link.split('/diseases')[0]
            full_link = response.urljoin(base_link + "/diseases/")
            yield scrapy.Request(url=full_link, callback=self.parse_disease_list, meta={'disease_type': disease_type, 'link': full_link})

    def parse_disease_list(self, response):
        disease_type = response.meta['disease_type']
        diseases = response.css('div.search-result.border div.content')
        for disease in diseases:
            disease_name = disease.css('a.bold span::text').get().strip()
            disease_link = disease.css('a.bold::attr(href)').get()
            full_disease_link = response.urljoin(disease_link)
            yield scrapy.Request(url=full_disease_link, callback=self.parse_disease_details, meta={'disease_type': disease_type, 'disease_name': disease_name})

    def parse_disease_details(self, response):
        disease_type = response.meta['disease_type']
        disease_name = response.meta['disease_name']
        
        # Extracting data for each section
        description = response.css('div#description').xpath('string()').get().strip()
        symptoms_title = response.css('div#symptoms > h2::text').get()
        symptoms = '\n'.join(response.css('div#symptoms ul li::text').getall())
        causes_and_risk_factors_title = response.css('div#causesandriskfactors > h2::text').get()
        causes_and_risk_factors = '\n'.join(response.css('div#causesandriskfactors ul li::text').getall())
        complications_title = response.css('div#complications > h2::text').get()
        complications = '\n'.join(response.css('div#complications ul li::text').getall())
        diagnosis_title = response.css('div#diagnosis > h2::text').get()
        diagnosis = '\n'.join(response.css('div#diagnosis ul li::text').getall())
        treatments_and_drugs_title = response.css('div#treatementsanddrugs > h2::text').get()
        treatments_and_drugs = '\n'.join(response.css('div#treatementsanddrugs p::text').getall())
        prevention_title = response.css('div#prevention > h2::text').get()
        prevention = '\n'.join(response.css('div#prevention ul li::text').getall())

        content = f"{disease_type}\n{disease_name}\n{description}\n\n{symptoms_title}:\n{symptoms}\n\n{causes_and_risk_factors_title}:\n{causes_and_risk_factors}\n\n{complications_title}:\n{complications}\n\n{diagnosis_title}:\n{diagnosis}\n\n{treatments_and_drugs_title}:\n{treatments_and_drugs}\n\n{prevention_title}:\n{prevention}"

        yield {
            'disease_type': disease_type,
            'disease_name': disease_name,
            'content': content
        }


# Create a process to run the spider
process = CrawlerProcess(settings={
    'FEEDS': {
        'disease_details2.json': {
            'format': 'json',
            'encoding': 'utf8',
            'overwrite': True,
        },
    },
})

# Run the spider
process.crawl(HealthSpider)
process.start()

# Load and display the data from the JSON file
with open('disease_details2.json', 'r', encoding='utf8') as f:
    disease_details = json.load(f)

# Display the extracted data
for item in disease_details:
    print(item['content'])


In [6]:
import pandas as pd
import json

# Load your JSON data with utf-8 encoding
with open('/kaggle/input/disease/disease_details1.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# Create a DataFrame
df = pd.DataFrame(data)

# Add an empty column named "score"
df = df.assign(score='')

# Display the first few rows
df.head()


,disease_type,disease_name,content,score
0,تسوس الأسنان,امراض الاغشية المخاطية للفم,تسوس الأسنان\nامراض الاغشية المخاطية للفم\nتغط...,
1,تسوس الأسنان,امراض التهاب اللب,تسوس الأسنان\nامراض التهاب اللب\nقد يكون الالت...,
2,تسوس الأسنان,عيوب خلقية لجوف الفم,تسوس الأسنان\nعيوب خلقية لجوف الفم\nيوجد للفم ...,
3,مرض السكري,حساسية الإنسولين,مرض السكري\nحساسية الإنسولين\nحتى نهاية سنوات ...,
4,تسوس الأسنان,ساركوما كابوزي,تسوس الأسنان\nساركوما كابوزي\nساركوما كابوزي ع...,


In [7]:
import pandas as pd
import json

# Define the score_text function
def score_text(text):
    # Define your scoring rules here
    score = 0
    
    # Keywords related to symptoms
    symptoms_keywords = ['ألم', 'عرض', 'علامة', 'ظهور', 'تشعر', 'تشعرين', 'يشعر', 'يعاني', 'يعانون', 'يعاني من', 'يعانون من', 'أعراض جانبية', 'تأثيرات جانبية', 'تغييرات في الجسم', 'وظائف غير طبيعية', 'مشاكل صحية', 'انزعاج', 'إزعاج', 'وجه', 'رأس', 'رقبة', 'صدر', 'ظهر', 'بطن', 'ذراع', 'ساق', 'مفصل', 'عضلة', 'جلد', 'شعر', 'أظافر', 'فم', 'لسان', 'أسنان', 'لثة', 'عيون', 'أذنين', 'أنف', 'شديد', 'حاد', 'خفيف', 'متوسط', 'مستمر', 'متقطع', 'مزمن', 'لا يطاق', 'غير محتمل', 'مُنهِك', 'مُؤلم', 'يُعيق', 'يُضعف', 'يُسبب إعاقة', 'حرقة', 'وخز', 'خدر', 'حكة', 'ألم غامض', 'ضعف', 'تشنج', 'غثيان', 'إقياء', 'إسهال', 'إمساك', 'سعال', 'ضيق تنفس', 'صعوبة في البلع', 'دوار', 'إغماء', 'نزيف', 'طفح جلدي', 'احمرار', 'تورم', 'جفاف', 'تقشر', 'تشقق', 'حكة', 'ألم عند اللمس', 'حساسية للمس', 'حساسية للحرارة', 'حساسية للبرودة', 'حساسية للضوء', 'حساسية للصوت', 'فقدان حاسة التذوق', 'فقدان حاسة الشم', 'قلق', 'اكتئاب', 'صداع', 'إرهاق', 'عصبية', 'توتر', 'اضطراب', 'مشاكل في النوم', 'ارتباك', 'تهيج', 'عدوانية', 'هلوسة', 'أوهام', 'فقدان التركيز', 'صعوبة في التفكير', 'صعوبة في اتخاذ القرارات', 'تغييرات في المزاج', 'سلوكيات غير طبيعية', 'ارتفاع درجة الحرارة', 'قشعريرة', 'حمى', 'قشعريرة', 'إفرازات مهبلية غير طبيعية', 'آلام الحوض', 'نزيف بين الدورات', 'نزيف بعد انقطاع الطمث', 'صعوبة التبول', 'إلحاح بولي', 'سلس البول', 'دم في البول', 'إفرازات قطنية بيضاء من المهبل', 'حرقان عند التبول', 'فرط التبول', 'تبول ليلي', 'ضعف الانتصاب', 'قلة الشهية']
    for keyword in symptoms_keywords:
        if keyword in text:
            score += 1
  
    # Keywords related to causes and risk factors
    causes_and_risk_factors_keywords = ['سبب', 'مسبب', 'عامل', 'يسبب', 'يؤدي إلى', 'يزيد من خطر', 'يقلل من خطر', 'مرض', 'عدوى', 'فيروس', 'بكتيريا', 'فطر', 'طفيلي', 'ورم', 'سرطان', 'خلل', 'اضطراب', 'حالة', 'وراثي', 'تاريخ عائلي', 'غذاء', 'تمارين', 'سمنة', 'تدخين', 'كحول', 'مخدرات', 'إجهاد', 'قلة نوم', 'تلوث', 'مواد كيميائية', 'إشعاع', 'لدغة', 'حشرة', 'إصابة', 'حادث', 'كسر', 'جرح', 'عمر', 'عوامل نفسية', 'ضغط عصبي', 'عوامل اجتماعية اقتصادية', 'فقر', 'انعدام الأمن الغذائي', 'إدمان الكحول والمخدرات', 'التعرض للعنف', 'التعرض للسموم', 'عوامل وراثية', 'طفرات جينية', 'عيوب خلقية', 'خلل التوازن الهرموني', 'أمراض المناعة الذاتية', 'استعداد وراثي', 'استهلاك الأدوية', 'التاريخ الطبي الشخصي', 'أمراض سابقة', 'الحمل', 'الرضاعة الطبيعية', 'سن اليأس', 'الجنس', 'العرق', 'النشاط البدني', 'تعرض أشعة الشمس']
    for keyword in causes_and_risk_factors_keywords:
        if keyword in text:
            score += 1
    
    # Keywords related to complications
    complications_keywords = ['مضاعفات', 'مضاعفة', 'يعقد', 'يؤدي إلى', 'يزيد من خطر', 'إعاقة', 'ضعف', 'فقدان وظيفة', 'اكتئاب', 'قلق', 'ألم مزمن', 'انتشار', 'وفاة', 'يهدد الحياة', 'تلف دائم', 'فشل عضوي', 'غيبوبة', 'عجز دائم', 'تشوهات', 'ندبات', 'التهاب مزمن', 'اضطرابات نفسية', 'اعتمادية على العلاج', 'انتكاس المرض', 'موت الأنسجة', 'نقص المناعة', 'سرطان ثانوي']
    for keyword in complications_keywords:
        if keyword in text:
            score += 1
    
    # Keywords related to diagnosis
    diagnosis_keywords = ['تشخيص', 'تشخيصي', 'يحدد', 'يكشف عن', 'يقيّم', 'تاريخ طبي', 'أعراض', 'فحص جسدي', 'فحص سريري', 'تحليل', 'فحص', 'تصوير', 'أشعة سينية', 'أشعة مقطعية', 'تصوير بالرنين المغناطيسي', 'تحليل دم', 'تحليل بول', 'خزعة', 'تنظير داخلي', 'تخطيط كهربية القلب', 'تخطيط كهربية الدماغ', 'قياس ضغط الدم', 'قياس السكر', 'اختبارات وظائف الكلى', 'اختبارات وظائف الكبد', 'اختبارات الحساسية', 'اختبارات الحمض النووي', 'فحص الحمض النووي', 'فحص جيني']
    for keyword in diagnosis_keywords:
        if keyword in text:
            score += 1
    
    # Keywords related to treatments and drugs
    treatments_and_drugs_keywords = ['علاج', 'علاج طبي', 'دواء', 'جرعة', 'جراحة', 'إشعاع', 'علاج طبيعي', 'علاج إشعاعي', 'علاج كيماوي', 'علاج بالهرمونات', 'علاج بالضوء', 'علاج سلوكي معرفي', 'علاج بالوخز بالإبر', 'علاج بالاسترخاء', 'علاج بالفن', 'علاج بالحمية الغذائية', 'علاج بديل', 'علاج تكميلي', 'جراحة من الحد الأدنى', 'جراحة تنظيرية', 'علاج بالخلايا الجذعية', 'علاج بالجينات', 'العلاج المناعي', 'المضادات الحيوية', 'مضادات الفيروسات', 'مضادات الفطريات', 'مضادات الطفيليات', 'مسكنات الألم', 'خافضات الحرارة', 'مضادات الالتهاب', 'مضادات الا']
    for keyword in treatments_and_drugs_keywords:
        if keyword in text:
            score += 1
    
    # Keywords related to prevention
    prevention_keywords = ['الوقاية', 'التوقي', 'الوقاية من', 'التوقي من', 'تجنب', 'تقليل', 'تحسين', 'الحد من', 'الوقاية الأولية', 'التوعية', 'التثقيف', 'التوجيه', 'النصائح', 'التوجيهات', 'التدابير', 'التدابير الوقائية', 'الإجراءات الوقائية', 'تغيير نمط الحياة', 'النظام الغذائي الصحي', 'ممارسة الرياضة', 'تجنب التدخين', 'التقليل من تناول الكحول', 'التقليل من تناول المخدرات', 'النوم الجيد', 'التقليل من التوتر', 'تجنب التعرض للعوامل الملوثة', 'التقليل من التعرض للإشعاع', 'استخدام وسائل الوقاية', 'التطعيم', 'التطهير', 'التعقيم', 'تنظيف اليدين', 'تنظيف البيئة', 'التغذية السليمة', 'العناية بالصحة النفسية', 'التوجيهات الطبية', 'الفحوصات الدورية', 'الكشف المبكر']
    for keyword in prevention_keywords:
        if keyword in text:
            score += 1
    
    return score

# Apply the score_text function to the 'content' column and update the 'score' column
df['score'] = df['content'].apply(score_text)

# Display the updated DataFrame
df.head()

,disease_type,disease_name,content,score
0,تسوس الأسنان,امراض الاغشية المخاطية للفم,تسوس الأسنان\nامراض الاغشية المخاطية للفم\nتغط...,37
1,تسوس الأسنان,امراض التهاب اللب,تسوس الأسنان\nامراض التهاب اللب\nقد يكون الالت...,37
2,تسوس الأسنان,عيوب خلقية لجوف الفم,تسوس الأسنان\nعيوب خلقية لجوف الفم\nيوجد للفم ...,42
3,مرض السكري,حساسية الإنسولين,مرض السكري\nحساسية الإنسولين\nحتى نهاية سنوات ...,20
4,تسوس الأسنان,ساركوما كابوزي,تسوس الأسنان\nساركوما كابوزي\nساركوما كابوزي ع...,25


In [9]:
def normalize_score(df, column_name):
    # Find the minimum and maximum scores
    min_score = df[column_name].min()
    max_score = df[column_name].max()
    
    # Normalize the scores between 0 and 10
    df['score'] = (df[column_name] - min_score) * 10 / (max_score - min_score)
    
    return df

# Example usage:
# Assuming df is your DataFrame and 'score' is the column containing scores
# Replace 'score' with the actual column name in your DataFrame
df = normalize_score(df, 'score')
df.head(10)

,disease_type,disease_name,content,score
0,تسوس الأسنان,امراض الاغشية المخاطية للفم,تسوس الأسنان\nامراض الاغشية المخاطية للفم\nتغط...,5.384615
1,تسوس الأسنان,امراض التهاب اللب,تسوس الأسنان\nامراض التهاب اللب\nقد يكون الالت...,5.384615
2,تسوس الأسنان,عيوب خلقية لجوف الفم,تسوس الأسنان\nعيوب خلقية لجوف الفم\nيوجد للفم ...,6.153846
3,مرض السكري,حساسية الإنسولين,مرض السكري\nحساسية الإنسولين\nحتى نهاية سنوات ...,2.769231
4,تسوس الأسنان,ساركوما كابوزي,تسوس الأسنان\nساركوما كابوزي\nساركوما كابوزي ع...,3.538462
5,تسوس الأسنان,جفاف الفم,تسوس الأسنان\nجفاف الفم\nيُساعد اللعاب على منع...,5.538462
6,تساقط الشعر,تساقط الشعر,تساقط الشعر\nتساقط الشعر\nأي شخص يُلاحظ أن شعر...,3.692308
7,تسوس الأسنان,رائحة الفم,تسوس الأسنان\nرائحة الفم\nيعاني العديد من الأش...,3.384615
8,الحمل المنتبذ,الولادة المبكرة,الحمل المنتبذ\nالولادة المبكرة\nيستمر الحمل ال...,4.461538
9,الحمل المنتبذ,الإجهاض,الحمل المنتبذ\nالإجهاض\nالإجهاض هو الفقد التلق...,7.076923


In [11]:
import re
import pandas as pd

# Define the basic_cleaning function
def basic_cleaning(text):
    text = str(text)  # Convert to string to handle non-string inputs
    text = text.lower()  # Lowercasing
    text = re.sub(r'{.*?}', '', text)   # Remove CSS code within curly braces
    # Removing all except alphanumeric characters, spaces, and specified characters
    text = re.sub(r'[^\w\s\-\+\*/!()\[\]{}<>]', '', text)
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Remove CSS
    text = re.sub(r'<style[^>]*>[\s\S]+?</style>', '', text)
    # Remove '\n' and replace with space
    text = text.replace('\n', ' ')
    # Remove non-Arabic characters and non-Arabic numbers
    text = re.sub(r'[^\u0621-\u064A0-9\s\-\+\*/!\[\]{}<>]', '', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Fill any NaN values with an empty string
df.fillna('', inplace=True)

In [33]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
import nltk
nltk.download('popular')  # This will download the most common resources including wordnet


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /usr/share/n

True

In [38]:

def tokenize(text):
    return word_tokenize(text)

def remove_stopwords(tokens):
    stop_words = set(stopwords.words('arabic'))
    return [word for word in tokens if word not in stop_words]

def stem_tokens(tokens):
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in tokens]

def lemmatize_tokens(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in tokens]

def preprocess_text(df, text_column):
    df['cleaned_text'] = df[text_column].apply(basic_cleaning)
    df['tokens'] = df['cleaned_text'].apply(tokenize)
    df['tokens'] = df['tokens'].apply(remove_stopwords)
    df['stemmed_tokens'] = df['tokens'].apply(lambda x: ' '.join(stem_tokens(x)))
    df['lemmatized_tokens'] = df['tokens'].apply(lambda x: ' '.join(lemmatize_tokens(x)))
    return df

preprocessed_df = preprocess_text(df, 'content')
preprocessed_df.head()

,disease_type,disease_name,content,score,cleaned_text,tokens,stemmed_tokens,lemmatized_tokens
0,تسوس الأسنان,امراض الاغشية المخاطية للفم,تسوس الأسنان\nامراض الاغشية المخاطية للفم\nتغط...,5.384615,تسوس الأسنان امراض الاغشية المخاطية للفم تغطي ...,"[تسوس, الأسنان, امراض, الاغشية, المخاطية, للفم...",تسوس الأسنان امراض الاغشية المخاطية للفم تغطي ...,تسوس الأسنان امراض الاغشية المخاطية للفم تغطي ...
1,تسوس الأسنان,امراض التهاب اللب,تسوس الأسنان\nامراض التهاب اللب\nقد يكون الالت...,5.384615,تسوس الأسنان امراض التهاب اللب قد يكون الالتها...,"[تسوس, الأسنان, امراض, التهاب, اللب, يكون, الا...",تسوس الأسنان امراض التهاب اللب يكون الالتهاب ا...,تسوس الأسنان امراض التهاب اللب يكون الالتهاب ا...
2,تسوس الأسنان,عيوب خلقية لجوف الفم,تسوس الأسنان\nعيوب خلقية لجوف الفم\nيوجد للفم ...,6.153846,تسوس الأسنان عيوب خلقية لجوف الفم يوجد للفم وظ...,"[تسوس, الأسنان, عيوب, خلقية, لجوف, الفم, يوجد,...",تسوس الأسنان عيوب خلقية لجوف الفم يوجد للفم وظ...,تسوس الأسنان عيوب خلقية لجوف الفم يوجد للفم وظ...
3,مرض السكري,حساسية الإنسولين,مرض السكري\nحساسية الإنسولين\nحتى نهاية سنوات ...,2.769231,مرض السكري حساسية الإنسولين حتى نهاية سنوات ال...,"[مرض, السكري, حساسية, الإنسولين, نهاية, سنوات,...",مرض السكري حساسية الإنسولين نهاية سنوات الثمان...,مرض السكري حساسية الإنسولين نهاية سنوات الثمان...
4,تسوس الأسنان,ساركوما كابوزي,تسوس الأسنان\nساركوما كابوزي\nساركوما كابوزي ع...,3.538462,تسوس الأسنان ساركوما كابوزي ساركوما كابوزي عبا...,"[تسوس, الأسنان, ساركوما, كابوزي, ساركوما, كابو...",تسوس الأسنان ساركوما كابوزي ساركوما كابوزي عبا...,تسوس الأسنان ساركوما كابوزي ساركوما كابوزي عبا...


In [40]:
#importing the libraries for encoding
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
import numpy as np

In [44]:
print(preprocessed_df['lemmatized_tokens'][983])

العيون حول الحول حول العيون حالة يلتقي محورا الرؤية العينين نقطة واحدة وقد يظهر الحول النظر الأمام الجانبين وقد يكون حالة ثابتة مستديمة متغيرة أوقات مختلفة أشكال الحول يتم التمييز عادة شكلين أساسيين الحول 1 الحول التصاحبي وهي الحالة تكون زاوية الحول مدى انحراف العين ثابتة اتجاهات النظر 2 الحول اللا تصاحبي وهي الحالة تختلف زاوية الحول باختلاف اتجاه النظر الشكل ينتج شلل واحدة العضلات تتحكم بحركات العين اتجاه انحراف العين يحدد نوع وشكل الحول حالة القبل الحول الداخلي الحول الإنسي يكون انحراف العين الداخل باتجاه الأنف حالة الخزر الحول الخارجي الحول الوحشي يكون انحراف العين الخارج باتجاه الأذن حالة الحول الفوقاني الحول المتجانس الأعلى يكون انحراف العين الأعلى حالة الحول التحتاني الحول المتجانس الأسفل يكون انحراف العين الأسفل يظهر حول العيون المراحل العمرية رغم معظم حالات الحول تظهر مرحلة الطفولة فالحول الداخلي الأطفال يظهر الأشهر الأولى عمر الطفل الحول الإنسي التكيفي فينشأ سن سنتين وثلاث سنوات ناتج الإصابة بمد البصر الحول يظهر التقدم السن يسمى الحول المكتسب الحالات الحول تكون نتيجة ضعف الرؤي

In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, Bidirectional, GRU, LSTM, Dense

In [50]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Assuming your DataFrame is named preprocessed_df
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_df['lemmatized_tokens'])
sequences = tokenizer.texts_to_sequences(preprocessed_df['lemmatized_tokens'])

# Pad the sequences
max_sequence_length = max([len(seq) for seq in sequences])
X = pad_sequences(sequences, maxlen=max_sequence_length)

# Prepare the labels
y = preprocessed_df['score'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [52]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

def create_rnn_model(input_shape):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=input_shape))
    model.add(SimpleRNN(128))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

rnn_model = create_rnn_model(X_train.shape[1])
rnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10


I0000 00:00:1733260732.438045     752 service.cc:145] XLA service 0x7fd6f0005b10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733260732.438107     752 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


 1/29 ━━━━━━━━━━━━━━━━━━━━ 1:16 3s/step - loss: 20.7331 - mae: 4.4050

I0000 00:00:1733260733.496637     752 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


29/29 ━━━━━━━━━━━━━━━━━━━━ 7s 144ms/step - loss: 14.7561 - mae: 3.5225 - val_loss: 2.1022 - val_mae: 1.2143
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 106ms/step - loss: 2.0134 - mae: 1.1344 - val_loss: 1.5732 - val_mae: 0.9819
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 106ms/step - loss: 1.5805 - mae: 0.9762 - val_loss: 1.5621 - val_mae: 0.9843
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 107ms/step - loss: 1.5813 - mae: 0.9653 - val_loss: 1.5591 - val_mae: 0.9810
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 106ms/step - loss: 1.2898 - mae: 0.8643 - val_loss: 1.5714 - val_mae: 0.9730
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 106ms/step - loss: 1.1629 - mae: 0.8002 - val_loss: 1.5645 - val_mae: 0.9706
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 106ms/step - loss: 0.6967 - mae: 0.6061 - val_loss: 1.6205 - val_mae: 0.9712
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 106ms/step - loss: 0.3048 - mae: 0.3878 - val_loss: 1.7051 - val_mae: 0.9927
Epoch 9/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 106ms/step - loss: 0.1315

In [53]:
from tensorflow.keras.layers import Bidirectional

def create_bidirectional_rnn_model(input_shape):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=input_shape))
    model.add(Bidirectional(SimpleRNN(128)))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

bidirectional_rnn_model = create_bidirectional_rnn_model(X_train.shape[1])
bidirectional_rnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 10s 252ms/step - loss: 9.7206 - mae: 2.5788 - val_loss: 1.6641 - val_mae: 0.9850
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 204ms/step - loss: 1.6274 - mae: 0.9974 - val_loss: 1.5337 - val_mae: 0.9753
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 204ms/step - loss: 1.5121 - mae: 0.9331 - val_loss: 1.5277 - val_mae: 0.9644
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 204ms/step - loss: 1.4942 - mae: 0.9247 - val_loss: 1.5412 - val_mae: 0.9548
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 204ms/step - loss: 1.0849 - mae: 0.7670 - val_loss: 1.4926 - val_mae: 0.9536
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 205ms/step - loss: 0.7552 - mae: 0.6087 - val_loss: 1.4827 - val_mae: 0.9502
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 204ms/step - loss: 0.3943 - mae: 0.4222 - val_loss: 1.4914 - val_mae: 0.9492
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 204ms/step - loss: 0.1610 - mae: 0.2573 - val_loss: 1.5040 - val_mae: 0.9537
Epoch 9/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 204ms/step - l

In [55]:
from tensorflow.keras.layers import GRU

def create_gru_model(input_shape):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=input_shape))
    model.add(GRU(128))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

gru_model = create_gru_model(X_train.shape[1])
gru_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 77ms/step - loss: 14.6202 - mae: 3.4601 - val_loss: 1.9109 - val_mae: 1.1068
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - loss: 2.1084 - mae: 1.1693 - val_loss: 1.5095 - val_mae: 0.9565
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - loss: 0.9732 - mae: 0.7506 - val_loss: 1.3575 - val_mae: 0.8988
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - loss: 0.4838 - mae: 0.5014 - val_loss: 1.4735 - val_mae: 0.9350
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - loss: 0.1982 - mae: 0.3084 - val_loss: 1.5045 - val_mae: 0.9485
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - loss: 0.1104 - mae: 0.2227 - val_loss: 1.5484 - val_mae: 0.9660
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - loss: 0.0681 - mae: 0.1690 - val_loss: 1.5683 - val_mae: 0.9714
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - loss: 0.0673 - mae: 0.1481 - val_loss: 1.5944 - val_mae: 0.9811
Epoch 9/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - loss: 0.06

In [56]:
from tensorflow.keras.layers import LSTM

def create_lstm_model(input_shape):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=input_shape))
    model.add(LSTM(128))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

lstm_model = create_lstm_model(X_train.shape[1])
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 78ms/step - loss: 12.7634 - mae: 3.0881 - val_loss: 1.5491 - val_mae: 0.9848
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - loss: 1.8028 - mae: 1.0610 - val_loss: 1.5356 - val_mae: 0.9661
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - loss: 1.5516 - mae: 0.9373 - val_loss: 1.4771 - val_mae: 0.9540
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - loss: 1.2660 - mae: 0.8556 - val_loss: 1.4245 - val_mae: 0.9505
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - loss: 0.6739 - mae: 0.6027 - val_loss: 1.4951 - val_mae: 0.9812
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - loss: 0.2455 - mae: 0.3630 - val_loss: 1.4872 - val_mae: 0.9682
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - loss: 0.1205 - mae: 0.2373 - val_loss: 1.5572 - val_mae: 0.9979
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - loss: 0.0517 - mae: 0.1492 - val_loss: 1.5566 - val_mae: 0.9858
Epoch 9/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - loss: 0.04

In [57]:
# Define the function to evaluate language models
def evaluate_language_models(models, X_test, y_test):
    evaluation_results = {}
    
    for model_name, model in models.items():
        # Evaluate the model using standard metrics
        loss, mae = model.evaluate(X_test, y_test)
        evaluation_results[model_name] = {'Loss': loss, 'MAE': mae}
    
    return evaluation_results

# Define your models and their names
models = {'Simple RNN': rnn_model, 'Bidirectional RNN': bidirectional_rnn_model, 'GRU': gru_model, 'LSTM': lstm_model}

# Evaluate the language models
evaluation_results = evaluate_language_models(models, X_test, y_test)

# Print the evaluation results
for model_name, metrics in evaluation_results.items():
    print(f"{model_name} - Loss: {metrics['Loss']}, MAE: {metrics['MAE']}")


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 1.5568 - mae: 0.9706
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 1.4344 - mae: 0.9325
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 1.5156 - mae: 0.9499
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 1.4038 - mae: 0.9352
Simple RNN - Loss: 1.6529250144958496, MAE: 0.9963415265083313
Bidirectional RNN - Loss: 1.517898440361023, MAE: 0.9540959596633911
GRU - Loss: 1.638801097869873, MAE: 0.9857399463653564
LSTM - Loss: 1.5648398399353027, MAE: 0.9905991554260254
